In [2]:
!pip install openai python-docx python-dotenv

In [3]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from docx import Document

In [4]:
class ReqDoc:
    def __init__(self, file_path):
        self.file_path = file_path

    def extract(self):
        """
        Reads the content of a .docx file and returns the paragraphs as a list of strings.
        """
        try:
            # Check if the file exists
            if not os.path.exists(self.file_path):
                raise FileNotFoundError(f"The file {self.file_path} was not found.")

            # Attempt to open and read the document
            doc = Document(self.file_path)
            text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
            return text

        except FileNotFoundError as fnf_error:
            print(fnf_error)
            return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None


In [5]:
# Initialize and constants
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj') and len(api_key)>10:
    print("API key looks good!")
else:
    print("There might be a problem with your API key. Please check!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good!


In [6]:
#Set up system prompt for extracting just the requirements from the document

req_doc_system_prompt = "You are provided with a complete requirements specifications document. \
You are able to decide which content from that document are related to actual requirements, identify each requirement as \
functional or non-functional and list them all.\n"
req_doc_system_prompt += "You should respond in JSON as in this example:"
req_doc_system_prompt += """
{
    "requirements": [
        {"RequirementNo": "FR-01", "Requirement Description": "description of this functional requirement goes here"},
        {"RequirementNo": "FR-02": "Requirement Description": "description of this functional requirement goes here"},
        {"RequirementNo": "NFR-01": "Requirement Description": "description of this non-functional requirement goes here"},
        {"RequirementNo": "NFR-02": "Requirement Description": "description of this non-functional requirement goes here"}
    ]
}
"""

In [7]:
#Set up user prompt, sending in the requirements doc as input and calling the ReqDoc.extract function. Key to note here is the explicit instructions to
#respond in JSON format.

def req_doc_user_prompt(doc):
    user_prompt = "Here is the contents from a requirement document.\n"
    user_prompt += f"{doc.extract()} \n"
    user_prompt += "Please scan through the document and extract only the  actual requirements. For example, ignore sections or \
paragraphs such as Approvers, table of contents and similar sections which are not really requirements.\
You must respond in a JSON format"
    user_prompt = user_prompt[:25_000] # Truncate if more than 25,000 characters
    return user_prompt



In [8]:
#Function to call chatgpt-4o-mini model with the user and system prompts set above and returning the json formatted result obtained from chatgpt

def get_requirements(doc):
    reqdoc = ReqDoc(doc)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": req_doc_system_prompt},
            {"role": "user", "content": req_doc_user_prompt(reqdoc)}
        ],
      response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [9]:
#Uncomment and run this if you want to see the extracted requriements in json format.
get_requirements("reqdoc.docx")

{'requirements': [{'RequirementNo': 'FR-01',
   'Requirement Description': 'Users can submit feedback via an online form.'},
  {'RequirementNo': 'FR-02',
   'Requirement Description': 'Feedback can be categorized by type (e.g., product, service).'},
  {'RequirementNo': 'FR-03',
   'Requirement Description': 'Administrators can generate detailed reports.'},
  {'RequirementNo': 'NFR-01',
   'Requirement Description': 'The system must support at least 10,000 concurrent users.'},
  {'RequirementNo': 'NFR-02',
   'Requirement Description': 'The feedback portal must load within 3 seconds for all pages.'},
  {'RequirementNo': 'NFR-03',
   'Requirement Description': 'Data must be encrypted during transmission and at rest.'}]}

### Next, we will make another call to gpt-4o-mini

In [11]:
#Set up system prompt to ask for test cases in table format

system_prompt = "You are an assitant that receives a list of functional and non functional requirements in JSON format. You are the expert in generating unit test cases for each requirement. \
You will create as many different test cases as needed for each requirement and produce a result in a table. Order the table by requirement No. Provide clear details on test case pass criteria. \
The table will contain the following columns. \
1.S No\
2.Requirement No\
3.Requirement Description\
4.Test Case ID\
5.Test case summary\
6.Test case description\
7.Success criteria \n"

In [12]:
# Set up user prompt passing in the req doc file. This in turn will call the get_requirements function, which will make a call to chatgpt.

def get_testcase_user_prompt(reqdoc):
    user_prompt = "You are looking at the following list of requirements. \n"
    user_prompt += f"{get_requirements(reqdoc)}\n"
    user_prompt += "Prepare unit test cases for each of these requirements in a table and send that table as response. \n"
    user_prompt += user_prompt[:25000]
    return user_prompt

In [13]:
#This is the 2nd call to chatgpt to get test cases. display(Markdown) will take care of producing a neatly formatted table output.
def create_testcase_doc(reqdoc):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_testcase_user_prompt(reqdoc)}
          ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [18]:
#The final piece of code. Provide the uploaded requirements filename below.
create_testcase_doc("reqdoc.docx")

Below are the unit test cases for the provided list of functional and non-functional requirements, organized in a table format.

| S No | Requirement No | Requirement Description                                                | Test Case ID | Test Case Summary                                       | Test Case Description                                                                                                 | Success Criteria                                                 |
|------|----------------|----------------------------------------------------------------------|---------------|--------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------|
| 1    | FR-01          | Users can submit feedback via an online form.                       | TC_FR-01-01   | Valid Submission                                        | Verify that a valid user can successfully submit feedback through the online form.                                    | Feedback is submitted successfully and stored in the database.   |
| 2    | FR-01          | Users can submit feedback via an online form.                       | TC_FR-01-02   | Invalid Submission                                      | Verify that an error message is displayed when a user attempts to submit an empty feedback form.                      | Error message "Feedback cannot be empty" is displayed.         |
| 3    | FR-02          | Feedback can be categorized by type (e.g., product, service).      | TC_FR-02-01   | Categorization by Type                                  | Check that feedback can be categorized correctly when submitted.                                                      | Feedback is categorized correctly in the database.               |
| 4    | FR-02          | Feedback can be categorized by type (e.g., product, service).      | TC_FR-02-02   | Invalid Category Handling                                | Check that an error message is shown if a user selects an invalid category.                                           | Error message "Invalid category selected" is displayed.         |
| 5    | FR-03          | Administrators can generate detailed reports.                       | TC_FR-03-01   | Report Generation                                       | Verify that an administrator can generate a report successfully.                                                     | Report is generated and downloaded without errors.               |
| 6    | FR-03          | Administrators can generate detailed reports.                       | TC_FR-03-02   | Empty Report Generation                                  | Verify that appropriate message is shown when there are no feedback entries to report on.                             | Message "No feedback entries found" is displayed.                |
| 7    | NFR-01         | The system must support at least 10,000 concurrent users.          | TC_NFR-01-01  | Concurrent Users Load Test                              | Simulate 10,000 concurrent users accessing the system to check for performance issues.                                 | System remains operational without significant slowdowns.       |
| 8    | NFR-02         | The feedback portal must load within 3 seconds for all pages.     | TC_NFR-02-01  | Page Load Time Test                                    | Measure the time taken for each page of the feedback portal to load.                                                  | All pages load within 3 seconds.                                 |
| 9    | NFR-03         | Data must be encrypted during transmission and at rest.            | TC_NFR-03-01  | Data Transmission Encryption Test                       | Verify that data is encrypted during transmission using encryption protocols (e.g., TLS).                             | Data during transmission is encrypted and not readable in transit.  |
| 10   | NFR-03         | Data must be encrypted during transmission and at rest.            | TC_NFR-03-02  | Data at Rest Encryption Test                            | Check that stored feedback data in the database is encrypted and cannot be accessed in plain text.                    | Data in the database is encrypted and unreadable without decryption. |

This table includes a comprehensive set of test cases addressing each requirement with clear pass criteria for effective validation.